In [1]:
import pandas as pd
import glob
import os
import re
import math
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean


In [2]:
def metric(path, r_string, type_string, timing=False):
    all_files = glob.glob(os.path.join(path, "*"+r_string+type_string+"*.csv"))
    for file in all_files:
        if os.path.getsize(file) == 0:
            all_files.remove(file)
    # print("Running Metric")
    print(type(all_files))
    results = dict()
    results[type_string] = dict()
    results['reg'] = dict()
    results['time_'+type_string] = dict()
    results['time_reg'] = dict()
    results['CPD_'+type_string] = dict()
    results['CPD_REG'] = dict()
    gnn_results = dict()
    reg_results = dict()
    for gnn_metric in all_files:
        lf = gnn_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__"+type_string+"__")[0]
        # print(name)
    
        df = pd.read_csv(gnn_metric,index_col=None,header=0)
       
        gnn_results[name] = int(max(df['Iteration']))
        # print(df[].head())
    results[type_string] = gnn_results
    reg_results = dict()
    for gnn_metric in all_files:
        lf = gnn_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__"+type_string+"__")[0]
        # print(name)
        df = pd.read_csv(gnn_metric,index_col=None,header=0)
       
        reg_results[name] = sum(df['Time'])
    results['time_'+type_string] = reg_results
    # print(results)
    reg_results = dict()

    reg_results = dict()
    for gnn_metric in all_files:
        lf = gnn_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__"+type_string+"__")[0]
        # print(name)
        df = pd.read_csv(gnn_metric,index_col=None,header=0)
       
        reg_results[name] = max(df['CPD (ns)'])
    results['CPD_'+type_string] = reg_results
    # print(results)
    reg_results = dict()
    all_files = glob.glob(os.path.join(path, r_string+"*reg*.csv"))
    for file in all_files:
        if os.path.getsize(file) == 0:
            all_files.remove(file)
    for reg_metric in all_files:
        lf = reg_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__reg__")[0]
        # print(name)
        df = pd.read_csv(reg_metric,index_col=None,header=0)
       
        reg_results[name] = max(df['Iteration'])
        # print(df[].head())  
    results['reg'] = reg_results

    reg_results = dict()
    for reg_metric in all_files:
        lf = reg_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__reg__")[0]
        # print(name)
        df = pd.read_csv(reg_metric,index_col=None,header=0)
        reg_results[name] = sum(df['Time'])
    results['time_reg'] = reg_results

    reg_results = dict()
    for reg_metric in all_files:
        lf = reg_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__reg__")[0]
        # print(name)
        df = pd.read_csv(reg_metric,index_col=None,header=0)
        reg_results[name] = max(df['CPD (ns)'])
    results['CPD_REG'] = reg_results
        # print(df[].head())

    # print(results)
    my_df = pd.DataFrame.from_dict(results)
    # print(my_df)
    my_df['Reduction'] = (my_df['reg'].values - my_df[type_string].values)/my_df['reg'].values
    # my_df['Reduction']=  pd.Series(["{0:.2f}%".format(val * 100) for val in my_df['Reduction']], index = my_df.index)

    # print("{:2%}".format(my_df))
    my_df.dropna(inplace=True)
    print("Average: ", "{:.2%}".format(mean(my_df['Reduction'].values)))

    pd.set_option('expand_frame_repr', False)
    my_df['Reduction']=  pd.Series(["{0:.2f}%".format(val * 100) for val in my_df['Reduction']], index = my_df.index)
    my_df['time_reg']=  pd.Series(["{0:.2f}".format(val) for val in my_df['time_reg']], index = my_df.index)
    my_df['time_'+type_string]=  pd.Series(["{0:.2f}".format(val) for val in my_df['time_'+type_string]], index = my_df.index)
    my_df['CPD_REG']=  pd.Series(["{0:.2f}".format(val) for val in my_df['CPD_REG']], index = my_df.index)
    my_df['CPD_'+type_string]=  pd.Series(["{0:.2f}".format(val) for val in my_df['CPD_'+type_string]], index = my_df.index)
   
    print(my_df)

In [30]:
path = "/mnt/e/benchmarks/route_metrics/MCNC/"
r_string = "EArch*"
type_string = "gnn"
metric(path, r_string, type_string)

<class 'list'>
Average:  -45.62%
                 gnn  reg time_gnn time_reg CPD_gnn CPD_REG Reduction
EArch__alu4        8   13     8.47     1.74    5.77    5.74    38.46%
EArch__apex2      15   11     9.53     1.85    6.60    6.56   -36.36%
EArch__apex4      12   12     8.05     1.66    5.60    5.46     0.00%
EArch__bigkey      7    7     8.69     1.04    2.32    2.30     0.00%
EArch__clma        9   16    11.76    10.32   11.82   11.79    43.75%
EArch__des        18   17     8.10     3.25    5.31    5.10    -5.88%
EArch__diffeq      5    8     8.24     0.95    6.83    6.83    37.50%
EArch__dsip        9   16     9.17     2.23    2.74    2.73    43.75%
EArch__elliptic    8   13     9.45     3.88    8.76    8.76    38.46%
EArch__ex1010     10   13     9.89     4.75    7.72    7.71    23.08%
EArch__ex5p        9   11     8.12     1.30    5.58    5.58    18.18%
EArch__frisc      32    8     9.73     2.42   12.65   12.64  -300.00%
EArch__misex3     15   22     6.92     2.66    5.26    5.

In [4]:
path = "/benchmarks/route_metrics/TITAN/"
r_string = "strat*"
type_string = "init"
metric(path, r_string, type_string)

<class 'list'>
Average:  -6.34%
                          init  reg time_init time_reg CPD_init CPD_REG Reduction
stratixiv_arch__alu4      16.0   16      1.86     0.76    10.78    9.53     0.00%
stratixiv_arch__apex2     24.0   18      7.46     1.19    13.32   10.37   -33.33%
stratixiv_arch__apex4     20.0   18      7.61     1.04    11.78    9.49   -11.11%
stratixiv_arch__bigkey     8.0    9      0.49     4.52     7.78    7.78    11.11%
stratixiv_arch__clma      19.0   17     44.68     4.36    10.92    9.93   -11.76%
stratixiv_arch__des       12.0   12      2.28     8.16    12.33   12.43     0.00%
stratixiv_arch__diffeq    15.0   17      5.60     1.10     9.89    7.51    11.76%
stratixiv_arch__dsip      11.0   14      4.42     7.03     7.43    7.43    21.43%
stratixiv_arch__elliptic  22.0   22    443.52     6.53    18.28   10.11     0.00%
stratixiv_arch__ex1010    25.0   22     45.99     4.32    17.90   12.69   -13.64%
stratixiv_arch__ex5p      22.0   17      9.72     1.07    12.42   